In [4]:
import pandas as pd 
import numpy as np 

from sklearn import preprocessing
from sklearn import metrics

from sklearn.utils import resample

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.combine import SMOTEENN

import fancyimpute

import matplotlib.pyplot as plt 
import seaborn as sns

In [179]:
df= pd.read_csv("Training_Dataset.csv")

In [180]:
df.head()

,cm_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,mvar48,mvar49,mvar50,mvar51
0,1,Charge,0,106920,4,18,970,0.7969,0.1831,337236,...,0,0,0,0,0,0,1,0,0,0
1,2,Charge,2,0,4,19,1030,0.0678,0.0000,2918974,...,0,0,0,0,1,0,0,0,0,0
2,3,Charge,2,399168,4,35,327,0.7551,0.2986,498656,...,0,0,0,0,0,0,1,0,0,0
3,4,Charge,0,267300,4,274,1258,0.7969,0.5297,2480074,...,0,1,0,0,0,0,1,0,0,1
4,5,Charge,0,213840,6,1,0,2.3238,0.0000,2918974,...,0,0,0,0,1,0,0,1,0,0


In [181]:
spend = df.mvar36 + df.mvar37 + df.mvar38 + df.mvar39 +df.mvar32 + df.mvar33 + df.mvar34 + df.mvar35 +df.mvar28 + df.mvar29 + df.mvar30 + df.mvar31+df.mvar24 + df.mvar25 + df.mvar26 + df.mvar27+df.mvar28 + df.mvar29 + df.mvar30 + df.mvar31
df["spend"]=spend

In [182]:
income = df["mvar9"][df.mvar9>0]

In [183]:
df.mvar9.replace(to_replace=0, value=np.nan, inplace=True)

In [184]:
df.mvar3.replace(to_replace=0, value=np.nan, inplace=True)

In [185]:
df.drop(df.columns[16:49].values, axis=1, inplace=True)

In [186]:
df.drop("mvar1", axis=1, inplace=True)

In [187]:
df.mvar50 = df.mvar50*2
df.mvar51= df.mvar51*3

In [188]:
df["card_accepted"]= df.mvar49+df.mvar50+df.mvar51

In [189]:
card= df.card_accepted

In [190]:
df.drop(["mvar49","mvar50","mvar51"], axis=1, inplace=True)

In [191]:

df.spend = (df.spend- df.spend.mean())/df.spend.std()

In [192]:
df.describe()

,cm_key,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar13,mvar14,mvar15,spend,card_accepted
count,40000.00000,40000.000000,15646.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,3.839500e+04,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,4.000000e+04,40000.000000
mean,20000.50000,1.117150,116343.140803,3.224150,89.264475,906.684950,0.726782,0.167726,2.082067e+06,0.329925,3.384588,11.993125,1.046025,0.329175,1.543703e-15,0.427150
std,11547.14972,1.656335,103311.603420,2.324337,92.247710,921.135812,1.262927,0.206095,3.895671e+06,0.470191,1.890469,3.274286,0.422328,0.772550,1.000000e+00,0.891046
min,1.00000,0.000000,445.000000,0.000000,1.000000,0.000000,0.000100,0.000000,6.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,-1.320092e-01,0.000000
25%,10000.75000,0.000000,37422.000000,2.000000,16.000000,327.000000,0.106000,0.000000,1.476990e+05,0.000000,1.753100,10.000000,1.000000,0.000000,-1.189555e-01,0.000000
50%,20000.50000,0.000000,89812.000000,3.000000,53.000000,586.000000,0.382000,0.115500,8.720850e+05,0.000000,3.220350,12.000000,1.000000,0.000000,-9.581564e-02,0.000000
75%,30000.25000,2.000000,160380.000000,4.000000,134.000000,1197.000000,0.796900,0.268200,2.918974e+06,1.000000,5.011550,14.000000,1.000000,0.000000,-3.146557e-02,0.000000
max,40000.00000,11.000000,434629.000000,12.000000,355.000000,4830.000000,10.000000,0.883100,3.420702e+07,1.000000,7.001000,23.000000,9.000000,5.000000,1.228794e+02,3.000000


In [193]:
le2 = preprocessing.LabelEncoder()
df.mvar12 = le2.fit_transform(df.mvar12)

In [194]:
impute_missing = df.iloc[:,:-1]
impute_missing_cols = list(impute_missing)
filled_soft = fancyimpute.MICE().complete(np.array(impute_missing))
df = pd.DataFrame(filled_soft, columns = impute_missing_cols)

[MICE] Completing matrix with shape (40000, 16)
[MICE] Starting imputation round 1/110, elapsed time 0.005
[MICE] Starting imputation round 2/110, elapsed time 0.265
[MICE] Starting imputation round 3/110, elapsed time 0.341
[MICE] Starting imputation round 4/110, elapsed time 0.412
[MICE] Starting imputation round 5/110, elapsed time 0.500
[MICE] Starting imputation round 6/110, elapsed time 0.570
[MICE] Starting imputation round 7/110, elapsed time 0.663
[MICE] Starting imputation round 8/110, elapsed time 0.693
[MICE] Starting imputation round 9/110, elapsed time 0.773
[MICE] Starting imputation round 10/110, elapsed time 0.831
[MICE] Starting imputation round 11/110, elapsed time 0.868
[MICE] Starting imputation round 12/110, elapsed time 0.969
[MICE] Starting imputation round 13/110, elapsed time 1.054
[MICE] Starting imputation round 14/110, elapsed time 1.122
[MICE] Starting imputation round 15/110, elapsed time 1.202
[MICE] Starting imputation round 16/110, elapsed time 1.294
[

In [295]:
# taking log
df.mvar9 = np.log(df.mvar9)

# normalizing
# std_scale = preprocessing.StandardScaler().fit(df3.mvar9)
# df3.mvar9 = std_scale.transform(df3.mvar9)

In [296]:
df.mvar3 = (df.mvar3 - df.mvar3.mean())/df.mvar3.std()

In [297]:
df["card_accepted"]= card.astype(int)

In [298]:
df.columns

Index([u'mvar2', u'mvar3', u'mvar4', u'mvar5', u'mvar6', u'mvar7', u'mvar8',
       u'mvar9', u'mvar10', u'mvar11', u'mvar12', u'mvar13', u'mvar14',
       u'mvar15', u'spend', u'card_accepted'],
      dtype='object')

In [299]:
df.drop("cm_key", axis=1, inplace=True)

ValueError: labels ['cm_key'] not contained in axis

In [200]:
df3 , test = train_test_split(df, test_size=0.3)

In [201]:
df3_majority = df3[df3.card_accepted==0]
df3_supp = df3[df3.card_accepted==1]
df3_elite = df3[df3.card_accepted==2]
df3_credit = df3[df3.card_accepted==3]
df3_majority.shape

(21748, 16)

In [414]:
df3_majority_downsampled = resample(df3_majority,
                                    replace=False,
                                    n_samples=10000,
                                    random_state=123)

In [415]:
df3_supp_upsampled = resample(df3_supp,
                              replace=True,
                              n_samples=10000,
                              random_state=123)

In [416]:
df3_elite_upsampled = resample(df3_elite,
                               replace=True,
                               n_samples=10000,
                               random_state=123)

In [417]:
df3_credit_upsampled = resample(df3_credit,
                                replace=True,    
                                n_samples=10000,     
                                random_state=123)

In [418]:
df4 = pd.concat([df3_majority_downsampled, df3_supp_upsampled, df3_elite_upsampled, df3_credit_upsampled])
df4.shape

(40000, 16)

In [419]:
x_train = df4.ix[:, :-1]
y_train = df4.ix[:, -1]
x_test = test.ix[:, :-1]
y_test = test.ix[:, -1]

/home/apoorva/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.


In [420]:
rfc = RandomForestClassifier(n_estimators=300, class_weight="balanced", oob_score=True, max_features='log2')

In [421]:
rfc.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='log2',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [422]:
y_predicted = rfc.predict(x_test)

In [423]:
print(metrics.accuracy_score(y_test, y_predicted))
print(metrics.confusion_matrix(y_test, y_predicted))
print(metrics.classification_report(y_test, y_predicted))

0.714
[[8103  473  432  366]
 [ 772  175   33   40]
 [ 579    5  162   20]
 [ 643   21   48  128]]
             precision    recall  f1-score   support

          0       0.80      0.86      0.83      9374
          1       0.26      0.17      0.21      1020
          2       0.24      0.21      0.22       766
          3       0.23      0.15      0.18       840

avg / total       0.68      0.71      0.69     12000



In [424]:
xgb = XGBClassifier(
    n_estimators=300,
    max_depth=3,
    min_child_weight=1,
    objective= 'multi:softmax',
 )

In [425]:
xgb.fit(x_train, y_train)

XGBClassifier(base_score=0.5, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=300, nthread=-1, objective='multi:softprob', seed=0,
       silent=True, subsample=1)

In [426]:
y_predicted = xgb.predict(x_test)

In [427]:
print(metrics.accuracy_score(y_test, y_predicted))
print(metrics.confusion_matrix(y_test, y_predicted))
print(metrics.classification_report(y_test, y_predicted))

0.367083333333
[[2952 1928 2612 1882]
 [ 187  465  206  162]
 [  90   41  563   72]
 [ 148   87  180  425]]
             precision    recall  f1-score   support

          0       0.87      0.31      0.46      9374
          1       0.18      0.46      0.26      1020
          2       0.16      0.73      0.26       766
          3       0.17      0.51      0.25       840

avg / total       0.72      0.37      0.42     12000



In [428]:
test_df= pd.read_csv("Leaderboard_Dataset.csv")
test_df.head()

,cm_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar36,mvar37,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45
0,50001,Charge,2,0,4,18,460,0.2495,0.2310,180512,...,32886.89,11345.47,7914.21,7704.62,0,0,0,0,0,0
1,50002,Charge,2,0,0,45,381,1.9329,0.7196,11295211,...,104722.52,255544.59,116216.40,152673.00,1,0,0,0,0,0
2,50003,Charge,0,0,4,3,0,0.7969,0.0000,45161,...,19689.03,0.00,0.00,0.00,0,0,0,0,0,0
3,50004,Charge,1,18711,9,89,432,0.1016,0.5074,420972,...,13154.43,7671.62,3357.24,2651.22,0,1,0,0,0,0
4,50005,Charge,0,21384,4,7,0,0.7969,0.2310,880618,...,32071.78,15376.01,14390.97,0.00,0,0,0,0,0,0


In [429]:
test_df= pd.read_csv("Leaderboard_Dataset.csv")

In [430]:
spend = test_df.mvar36 + test_df.mvar37 + test_df.mvar38 + test_df.mvar39 +test_df.mvar32 + test_df.mvar33 + test_df.mvar34 + test_df.mvar35 +test_df.mvar28 + test_df.mvar29 + test_df.mvar30 + test_df.mvar31+test_df.mvar24 + test_df.mvar25 + test_df.mvar26 + test_df.mvar27+test_df.mvar28 + test_df.mvar29 + test_df.mvar30 + test_df.mvar31
test_df["spend"]=spend

test_df.mvar9.replace(to_replace=0, value=np.nan, inplace=True)

test_df.mvar3.replace(to_replace=0, value=np.nan, inplace=True)

test_df.drop(test_df.columns[16:46].values, axis=1, inplace=True)

test_df.drop("mvar1", axis=1, inplace=True)


test_df.spend = (test_df.spend- test_df.spend.mean())/test_df.spend.std()


le2 = preprocessing.LabelEncoder()
test_df.mvar12 = le2.fit_transform(test_df.mvar12)

impute_missing = test_df
impute_missing_cols = list(impute_missing)
filled_soft = fancyimpute.MICE().complete(np.array(impute_missing))
test_df = pd.DataFrame(filled_soft, columns = impute_missing_cols)

# taking log
test_df.mvar9 = np.log(test_df.mvar9)


test_df.mvar3 = (test_df.mvar3 - test_df.mvar3.mean())/test_df.mvar3.std()

[MICE] Completing matrix with shape (10000, 16)
[MICE] Starting imputation round 1/110, elapsed time 0.009
[MICE] Starting imputation round 2/110, elapsed time 0.299
[MICE] Starting imputation round 3/110, elapsed time 0.307
[MICE] Starting imputation round 4/110, elapsed time 0.317
[MICE] Starting imputation round 5/110, elapsed time 0.328
[MICE] Starting imputation round 6/110, elapsed time 0.340
[MICE] Starting imputation round 7/110, elapsed time 0.347
[MICE] Starting imputation round 8/110, elapsed time 0.356
[MICE] Starting imputation round 9/110, elapsed time 0.364
[MICE] Starting imputation round 10/110, elapsed time 0.372
[MICE] Starting imputation round 11/110, elapsed time 0.379
[MICE] Starting imputation round 12/110, elapsed time 0.394
[MICE] Starting imputation round 13/110, elapsed time 0.425
[MICE] Starting imputation round 14/110, elapsed time 0.435
[MICE] Starting imputation round 15/110, elapsed time 0.446
[MICE] Starting imputation round 16/110, elapsed time 0.467
[

In [431]:
test_df.drop("cm_key", axis=1, inplace=True)

In [432]:
test_df.columns

Index([u'mvar2', u'mvar3', u'mvar4', u'mvar5', u'mvar6', u'mvar7', u'mvar8',
       u'mvar9', u'mvar10', u'mvar11', u'mvar12', u'mvar13', u'mvar14',
       u'mvar15', u'spend'],
      dtype='object')

In [433]:
test_result= rfc.predict(test_df)

In [434]:
test_df2= pd.read_csv("Leaderboard_Dataset.csv")

In [441]:
rfc_proba= pd.DataFrame(rfc.predict_proba(test_df),  columns= ["r_0", "r_1", "r_2", "r_3"])

In [442]:
test_result_xgb= xgb.predict(test_df)

In [443]:
xgb_proba= pd.DataFrame(xgb.predict_proba(test_df), columns= ["x_0", "x_1", "x_2", "x_3"])

In [444]:
results= pd.DataFrame({
    "rfc":test_result,
    "xgb":test_result_xgb,
    "cm_key":test_df2.cm_key
})

In [445]:
results["equal"]= results.rfc==results.xgb

In [446]:
results= pd.concat([results, rfc_proba, xgb_proba], axis=1)

In [447]:
results["0"]= (results.r_0 + results.x_0)/2
results["1"]= (results.r_1 + results.x_1)/2
results["2"]= (results.r_2 + results.x_2)/2
results["3"]= (results.r_3 + results.x_3)/2


In [448]:
results["max"]= results.iloc[:,-4:].max(axis=1)

In [456]:
results= results.sort_values("max", ascending=False)
results.iloc[1,12]

0.12751750186085703

In [457]:
max_no =[]
for n in range(0, len(results)):
    if (results.iloc[n, -1]== results.iloc[n, 12]):
        max_no.append(0)
    if (results.iloc[n, -1]== results.iloc[n, 13]):
        max_no.append(1)
    if (results.iloc[n, -1]== results.iloc[n, 14]):
        max_no.append(2)
    if (results.iloc[n, -1]== results.iloc[n, 15]):
        max_no.append(3)

In [458]:
results["max_no"]= max_no

In [471]:
submission= results[results.max_no>0][:1000]

In [460]:
results_both= results[results.equal==True]

In [461]:
results_both= results_both[results_both.rfc>0]
results_both

,cm_key,rfc,xgb,equal,r_0,r_1,r_2,r_3,x_0,x_1,x_2,x_3,0,1,2,3,max,max_no
7923,57924,1,1,True,0.170000,0.823333,0.000000,0.006667,0.085035,0.910540,0.000070,0.004355,0.127518,0.866937,0.000035,0.005511,0.866937,1
8072,58073,1,1,True,0.190000,0.806667,0.000000,0.003333,0.078662,0.918314,0.000066,0.002958,0.134331,0.862491,0.000033,0.003146,0.862491,1
9391,59392,1,1,True,0.206667,0.783333,0.000000,0.010000,0.055583,0.938606,0.000051,0.005761,0.131125,0.860969,0.000025,0.007880,0.860969,1
514,50515,1,1,True,0.183333,0.786667,0.000000,0.030000,0.063597,0.928742,0.000074,0.007587,0.123465,0.857704,0.000037,0.018794,0.857704,1
4764,54765,1,1,True,0.213333,0.776667,0.000000,0.010000,0.095538,0.890615,0.000043,0.013804,0.154436,0.833641,0.000021,0.011902,0.833641,1
9045,59046,1,1,True,0.156667,0.823333,0.000000,0.020000,0.096812,0.831335,0.000255,0.071599,0.126739,0.827334,0.000127,0.045799,0.827334,1
5422,55423,1,1,True,0.216667,0.776667,0.003333,0.003333,0.130571,0.862910,0.000250,0.006270,0.173619,0.819789,0.001791,0.004801,0.819789,1
5210,55211,1,1,True,0.180000,0.813333,0.003333,0.003333,0.155292,0.825088,0.000222,0.019398,0.167646,0.819210,0.001778,0.011366,0.819210,1
6923,56924,1,1,True,0.216667,0.776667,0.000000,0.006667,0.109166,0.857168,0.000122,0.033544,0.162916,0.816917,0.000061,0.020106,0.816917,1
1276,51277,1,1,True,0.256667,0.730000,0.010000,0.003333,0.097399,0.896931,0.000053,0.005617,0.177033,0.813465,0.005027,0.004475,0.813465,1


In [472]:
submission = submission[["cm_key", "max_no"]]

In [473]:
submission.head()

,cm_key,max_no
7923,57924,1
8072,58073,1
9391,59392,1
514,50515,1
4764,54765,1


In [474]:
submission.max_no.replace(to_replace=1, value="Supp", inplace=True)
submission.max_no.replace(to_replace=2, value="Elite", inplace=True)
submission.max_no.replace(to_replace=3, value="Credit", inplace=True)

In [475]:
submission

,cm_key,max_no
7923,57924,Supp
8072,58073,Supp
9391,59392,Supp
514,50515,Supp
4764,54765,Supp
9045,59046,Supp
5422,55423,Supp
5210,55211,Supp
6923,56924,Supp
1276,51277,Supp


In [352]:
#submission= submission.reindex_axis(sorted(submission.columns, reverse=True), axis=1)

In [353]:
#submission= submission[:1000]

In [469]:
submission

,cm_key,rfc
7923,57924,Supp
8072,58073,Supp
9391,59392,Supp
514,50515,Supp
4764,54765,Supp
9045,59046,Supp
5422,55423,Supp
5210,55211,Supp
6923,56924,Supp
1276,51277,Supp


In [476]:
submission.to_csv("Reckless_Serenade_IITRoorkee_7.csv", header=False, index=False)

In [1]:
test_df2= pd.read_csv("Final_Dataset.csv")

NameError: name 'pd' is not defined